In [1]:
import torch
import torch.nn as nn
import math

Verificación de GPU en uso:

In [5]:
# Verificar si la GPU está disponible
if torch.cuda.is_available():
    print(f"GPU detectada: {torch.cuda.get_device_name(0)}")
    print(f"Memoria disponible: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("No se detectó GPU, se utilizará la CPU.")
    

tensor = torch.randn(1000).to('cuda')
print(tensor.device)

GPU detectada: NVIDIA GeForce GTX 1080
Memoria disponible: 8.50 GB
cuda:0


*Scaled Dot Product Attention:*
Calcula la atención entre los vectores de consulta (query), clave (key) y valor (value).

In [4]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, mask=None):
        d_k = Q.size(-1) # Escala el producto punto por la raíz cuadrada de d_k
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_k)

        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9) # Enmascara valores para evitar la atención en ciertas posiciones

        attention = torch.nn.functional.softmax(scores, dim=-1) # Aplica softmax para obtener los pesos de atención
        output = torch.matmul(attention, V) # Multiplica los pesos de atención por los valores
        return output, attention

*MultiHead Attention:*
Permite calcular múltiples cabezas de atención en paralelo.

In [20]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0
        self.d_k = d_model // num_heads # Tamaño de la dimensión de cada cabeza de atención

        self.num_heads = num_heads

        # Capas lineales para proyectar las consultas, claves y valores en las cabezas de atención
        self.query = nn.Linear(d_model, d_model)
        self.key = nn.Linear(d_model, d_model)
        self.value = nn.Linear(d_model, d_model)
        
        # Capa lineal para proyectar la salida de las cabezas de atención
        self.output = nn.Linear(d_model, d_model)

    def forward(self, Q, K, V, mask=None):
        batch_size = Q.size(0)

        # Proyecciones lineales y cambio de forma para aplicar atención por cada cabeza
        Q = self.query(Q).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = self.key(K).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = self.value(V).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)

        if mask is not None:
            mask = mask.unsqueeze(1)

        output, attention = ScaledDotProductAttention()(Q, K, V, mask)

        # Reorganización de la salida y consolidación de todas las cabezas en una sola representación
        output = output.transpose(1, 2).contiguous().view(batch_size, -1, self.num_heads * self.d_k)
        return self.output(output), attention

*FeedForward:* Red completamente conectada que ayuda a la transformación no lineal de las representaciones.

In [21]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(FeedForward, self).__init__()

        # Dos capas lineales con una activación ReLU intermedia
        self.linear1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(0.1)  # Dropout layer to prevent overfitting, with a probability of 0.1 (10%)  # 0.1 is a common dropout rate in Transformer models.
        self.linear2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        return self.linear2(self.dropout(torch.nn.functional.relu(self.linear1(x))))

*Transformer Layer:* Combina las capas de MultiHead Attention y Feed Forward con mecanismos de normalización y conexiones residuales.

In [22]:
class TransformerLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff):
        super(TransformerLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = FeedForward(d_model, d_ff)

        # Capas de normalización y dropout para mejorar estabilidad y evitar sobreajuste
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)

    def forward(self, x, mask=None):
        # Atención multicabeza con residual y normalización
        attn_output, _ = self.attention(x, x, x, mask)
        x = x + self.dropout1(attn_output)
        x = self.norm1(x)

        # Feed forward con residual y normalización
        ff_output = self.feed_forward(x)
        x = x + self.dropout2(ff_output)
        x = self.norm2(x)
        return x

*Transformer:* Apila múltiples capas de TransformerLayer para crear una arquitectura completa.

In [23]:
class Transformer(nn.Module):
    def __init__(self, input_dim, d_model, num_heads, d_ff, num_layers):
        super(Transformer, self).__init__()

        # Capa de embedding para convertir tokens en vectores densos
        self.embedding = nn.Embedding(input_dim, d_model)

        self.layers = nn.ModuleList([TransformerLayer(d_model, num_heads, d_ff) for _ in range(num_layers)])
        self.fc_out = nn.Linear(d_model, input_dim) # Capa de salida para proyectar de vuelta a la dimensión original
       
    def forward(self, x, mask=None):
        x = self.embedding(x)
        for layer in self.layers:
            x = layer(x, mask)
        return self.fc_out(x)

In [24]:
# Definición de hiperparámetros
input_dim = 10000  # Tamaño del vocabulario
d_model = 512
num_heads = 8
d_ff = 2048
num_layers = 6

model = Transformer(input_dim=input_dim, d_model=d_model, num_heads=num_heads, d_ff=d_ff, num_layers=num_layers)

# Crear un ejemplo de secuencia de entrada
input_sequence = torch.randint(0, input_dim, (2, 10))  # Secuencia de ejemplo

# Pasar la secuencia al modelo
output = model(input_sequence)
print("Dimensiones de la salida:", output.shape)

Dimensiones de la salida: torch.Size([2, 10, 10000])
